# KNN 연습

- user_id, session_id, clickout_cnt, action_ref

In [18]:


def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out


In [20]:
import math
import pandas as pd
import numpy as np
import sys
np.set_printoptions(threshold = sys.maxsize)

In [146]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
train = pd.read_csv('../data/train_v3.csv', sep = ',', encoding = 'utf-8')
df_item = pd.read_csv('../data/df_item_v2.csv')
#item_metadata = pd.read_csv('../data/item_metadata_v2.csv')

In [290]:
# 데이터 확인
#train.head(20)
#item_metadata.head(10)
#df_item

In [331]:
#sample 데이터 
small_train = train.loc[train['user_id']== '00RL8Z82B2Z1'].loc[train['session_id'] == 'aff3928535f48'] #확인필요
small_train

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
9,00RL8Z82B2Z1,aff3928535f48,1541037542,10,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [116]:
#list(map(int,small_train.loc[small_train['action_type'] =='interaction item image'].reference.value_counts().index.tolist()))# 

In [113]:
#small_train.loc[small_train['action_type'] =='clickout item'].reference.value_counts()  # 정답

In [114]:
#small_train.loc[small_train.impressions.notnull()].impressions

In [115]:
#small_train.loc[small_train.impressions.notnull()].impressions.apply(lambda x : string_to_array(x))  # 추천목록

In [337]:
# 이고!
# action_type = ['interaction item image', 'interaction item info', 'interaction item rating','interaction item deals', 'search for item']
action = list(map(int,small_train.loc[small_train['action_type'] =='interaction item image'].reference.value_counts().index.tolist()))#  # 행위
recommend =small_train.loc[small_train.impressions.notnull()].impressions.apply(lambda x : list(map(int,string_to_array(x))))  # 추천목록

# 구매한것
#109038, 1257342

print('user 가 상호작용한 item 목록')
print(action)
print('')
print('impression 목록')
print(recommend)


# recommend 의 index 를 click out 한 아이템으로 만들기!

user 가 상호작용한 item 목록
[109038, 666856]

impression 목록
13    [3400638, 1253714, 3367857, 5100540, 1088584, ...
15    [55109, 129343, 54824, 2297972, 109014, 125734...
Name: impressions, dtype: object


In [338]:
item_list = action  + recommend.iloc[0] + recommend.iloc[1] 
print(len(item_list))
print(len(set(item_list)))
print('상호작용한 item 과 추천목록에 뜬 item 목록')
item_list = list(set(item_list))
print(item_list)
print(len(item_list))

52
41
상호작용한 item 과 추천목록에 뜬 item 목록
[8444418, 10077318, 54804, 6806806, 1031578, 2050977, 666916, 2237222, 54824, 3909420, 55088, 3367857, 54833, 55091, 666936, 1431482, 3400638, 129343, 2595006, 55109, 2922310, 1088584, 9711560, 1166793, 6622154, 3095758, 1253714, 109013, 109014, 109018, 9132132, 54885, 666856, 1332971, 109038, 6339822, 2297972, 1041528, 1474297, 5100540, 1257342]
41


In [339]:
# item list sorting 인덱스 확인을 위해 정렬이 필요
item_list.sort()
item_list

[54804,
 54824,
 54833,
 54885,
 55088,
 55091,
 55109,
 109013,
 109014,
 109018,
 109038,
 129343,
 666856,
 666916,
 666936,
 1031578,
 1041528,
 1088584,
 1166793,
 1253714,
 1257342,
 1332971,
 1431482,
 1474297,
 2050977,
 2237222,
 2297972,
 2595006,
 2922310,
 3095758,
 3367857,
 3400638,
 3909420,
 5100540,
 6339822,
 6622154,
 6806806,
 8444418,
 9132132,
 9711560,
 10077318]

In [225]:
# 메타데이터에서 호텔리스트 뽑기
# 상호작용한 아이템만 (메타데이터)
df_item.loc[df_item['item_id'].isin(action)]
#df_item.loc[df_item['item_id'].isin(recommend.iloc[0])]


,item_id,properties,mean_price,min_price,max_price,var_price,std_price,n_impressions,n_interaction_item_info,n_interaction_item_image,n_interaction_item_rating,n_interaction_item_deals,n_search_for_item,n_clickout_item
246986,109038,Electric Kettle|Fan|Flatscreen TV|Beach Bar|La...,101.582515,50.0,568.0,2555.253015,50.549511,1018.0,7.0,204.0,2.0,7.0,NaN,59.0
423093,666856,Laundry Service|Family Friendly|WiFi (Public A...,67.987500,19.0,168.0,722.624844,26.881682,160.0,3.0,68.0,3.0,2.0,NaN,5.0


In [226]:
# 추천목록에 뜬 아이템만 (메타데이터)
df_item.loc[df_item['item_id'].isin(recommend.iloc[0])]

,item_id,properties,mean_price,min_price,max_price,var_price,std_price,n_impressions,n_interaction_item_info,n_interaction_item_image,n_interaction_item_rating,n_interaction_item_deals,n_search_for_item,n_clickout_item
43073,1431482,Hotel|Satisfactory Rating|3 Star|Self Catering...,93.340836,26.0,425.0,5825.787368,76.326846,311.0,4.0,105.0,NaN,2.0,NaN,14.0
96034,6339822,Hotel|Satisfactory Rating|3 Star|Luxury Hotel|...,70.264286,46.0,195.0,685.351582,26.179220,140.0,3.0,107.0,NaN,NaN,NaN,4.0
104211,3367857,Electric Kettle|Desk|From 2 Stars|Car Park|Lau...,233.926606,27.0,1014.0,53238.655164,230.735032,327.0,2.0,145.0,3.0,NaN,NaN,19.0
108575,2922310,Fan|Fridge|Washing Machine|Lift|Central Heatin...,97.793893,48.0,244.0,1103.247596,33.215171,131.0,2.0,25.0,NaN,1.0,NaN,6.0
112151,666916,WiFi (Public Areas)|Electric Kettle|Luxury Hot...,291.247059,87.0,2397.0,143755.927197,379.151589,255.0,NaN,17.0,NaN,NaN,NaN,2.0
112319,1088584,Balcony|WiFi (Public Areas)|Microwave|Hairdrye...,80.934615,38.0,308.0,2399.576494,48.985472,260.0,3.0,48.0,6.0,NaN,NaN,4.0
160150,3095758,From 2 Stars|Ironing Board|Free WiFi (Combined...,86.519231,58.0,231.0,565.839374,23.787378,156.0,NaN,NaN,NaN,NaN,NaN,1.0
177220,55088,Restaurant|Openable Windows|Radio|WiFi (Public...,128.052239,80.0,674.0,4254.661450,65.227766,134.0,NaN,32.0,NaN,NaN,NaN,2.0
201380,2595006,Hiking Trail|Television|Business Hotel|Shower|...,214.444444,99.0,564.0,12387.709877,111.300089,108.0,1.0,NaN,NaN,NaN,4.0,3.0
246986,109038,Electric Kettle|Fan|Flatscreen TV|Beach Bar|La...,101.582515,50.0,568.0,2555.253015,50.549511,1018.0,7.0,204.0,2.0,7.0,NaN,59.0


In [340]:
## (상호작용+추천목록들)하나로 합친 아이템들 (메타데이터)
print(len(df_item.loc[df_item['item_id'].isin(item_list)]))
x = df_item.loc[df_item['item_id'].isin(item_list)].set_index('item_id').sort_index().drop(['properties'], axis = 1) # 인덱스 정렬필요
x

41


,mean_price,min_price,max_price,var_price,std_price,n_impressions,n_interaction_item_info,n_interaction_item_image,n_interaction_item_rating,n_interaction_item_deals,n_search_for_item,n_clickout_item
item_id,,,,,,,,,,,,
54804,169.987124,69.0,731.0,6832.484813,82.658846,466.0,3.0,28.0,2.0,5.0,2.0,13.0
54824,108.008412,42.0,408.0,1946.801191,44.122570,951.0,8.0,409.0,4.0,2.0,NaN,87.0
54833,147.811321,90.0,350.0,1930.907796,43.942096,159.0,NaN,24.0,1.0,NaN,1.0,8.0
54885,160.839676,85.0,534.0,2570.728952,50.702356,1235.0,11.0,472.0,5.0,8.0,4.0,54.0
55088,128.052239,80.0,674.0,4254.661450,65.227766,134.0,NaN,32.0,NaN,NaN,NaN,2.0
55091,137.123711,20.0,247.0,1387.469232,37.248748,97.0,NaN,2.0,NaN,NaN,NaN,4.0
55109,190.576954,65.0,2839.0,39307.904030,198.262210,1241.0,14.0,553.0,5.0,9.0,3.0,35.0
109013,156.530233,70.0,905.0,8489.695598,92.139544,430.0,1.0,102.0,2.0,1.0,2.0,13.0
109014,148.615385,67.0,628.0,2953.594152,54.346979,442.0,5.0,258.0,4.0,5.0,1.0,28.0


In [341]:
#ndarry.nan_to_num
X = np.nan_to_num(x)
X


array([[1.69987124e+02, 6.90000000e+01, 7.31000000e+02, 6.83248481e+03,
        8.26588459e+01, 4.66000000e+02, 3.00000000e+00, 2.80000000e+01,
        2.00000000e+00, 5.00000000e+00, 2.00000000e+00, 1.30000000e+01],
       [1.08008412e+02, 4.20000000e+01, 4.08000000e+02, 1.94680119e+03,
        4.41225701e+01, 9.51000000e+02, 8.00000000e+00, 4.09000000e+02,
        4.00000000e+00, 2.00000000e+00, 0.00000000e+00, 8.70000000e+01],
       [1.47811321e+02, 9.00000000e+01, 3.50000000e+02, 1.93090780e+03,
        4.39420959e+01, 1.59000000e+02, 0.00000000e+00, 2.40000000e+01,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00, 8.00000000e+00],
       [1.60839676e+02, 8.50000000e+01, 5.34000000e+02, 2.57072895e+03,
        5.07023565e+01, 1.23500000e+03, 1.10000000e+01, 4.72000000e+02,
        5.00000000e+00, 8.00000000e+00, 4.00000000e+00, 5.40000000e+01],
       [1.28052239e+02, 8.00000000e+01, 6.74000000e+02, 4.25466145e+03,
        6.52277659e+01, 1.34000000e+02, 0.00000000e+00, 3.20

In [250]:
sim_practice = cosine_similarity(X )

In [251]:
sim_practice

array([[1.        , 0.9092471 , 0.99531743, 0.91532338, 0.99799839,
        0.99489352, 0.99844511, 0.99979491, 0.98819948, 0.99774262,
        0.94613165, 0.99874262, 0.97640086, 0.99344501, 0.96175419,
        0.99644714, 0.8945863 , 0.99879654, 0.99641896, 0.9798915 ,
        0.99651623, 0.99347996, 0.99917666, 0.31957816, 0.9946562 ,
        0.95168856, 0.6552145 , 0.99635745, 0.99011097, 0.93653537,
        0.99400468, 0.99730468, 0.99307473, 0.99933993, 0.96405386,
        0.99747485, 0.99706353, 0.99329402, 0.83900948, 0.98734408,
        0.99927167],
       [0.9092471 , 1.        , 0.91941145, 0.99944872, 0.89943687,
        0.91270119, 0.89276037, 0.90391936, 0.95089211, 0.92853025,
        0.99082752, 0.89568769, 0.96751293, 0.8700441 , 0.91102919,
        0.87875599, 0.9946221 , 0.9273869 , 0.94047897, 0.94385226,
        0.92603328, 0.94628916, 0.90249926, 0.63688669, 0.93076988,
        0.98771679, 0.71087489, 0.87819488, 0.93262102, 0.94384182,
        0.87288853, 0.88372

In [248]:
item_list.index(109038)

10

In [254]:
item_list
#action

[54804,
 54824,
 54833,
 54885,
 55088,
 55091,
 55109,
 109013,
 109014,
 109018,
 109038,
 129343,
 666856,
 666916,
 666936,
 1031578,
 1041528,
 1088584,
 1166793,
 1253714,
 1257342,
 1332971,
 1431482,
 1474297,
 2050977,
 2237222,
 2297972,
 2595006,
 2922310,
 3095758,
 3367857,
 3400638,
 3909420,
 5100540,
 6339822,
 6622154,
 6806806,
 8444418,
 9132132,
 9711560,
 10077318]

In [257]:
# 상호작용한 아이템 109038과 유사한 아이템인덱스 리스트 
item_list.index(109038) # 10 
a = list(np.argsort(-sim_practice)[10])  # 109038 와 유사한순서대로 아이템 인덱스리스트
print(a)

[10, 25, 3, 1, 16, 12, 34, 39, 21, 19, 8, 18, 29, 28, 24, 20, 17, 9, 2, 33, 40, 5, 0, 14, 7, 38, 4, 22, 11, 6, 31, 35, 36, 15, 27, 30, 13, 37, 32, 26, 23]


In [277]:
# 위 순서대로 item_id 를 리스트 안에 넣었다. (유사한 순서임)
# for  i in a:
#     print(item_list[i])
[item_list[i] for i in a]  # 인덱스리스트에서 호텔아이디를 가져옴
pd.Series([item_list[i] for i in a]) # 시리즈로 만들면서 시리즈 인덱스가 109038 과 비슷한 순서임

0       109038
1      2237222
2        54885
3        54824
4      1041528
5       666856
6      6339822
7      9711560
8      1332971
9      1253714
10      109014
11     1166793
12     3095758
13     2922310
14     2050977
15     1257342
16     1088584
17      109018
18       54833
19     5100540
20    10077318
21       55091
22       54804
23      666936
24      109013
25     9132132
26       55088
27     1431482
28      129343
29       55109
30     3400638
31     6622154
32     6806806
33     1031578
34     2595006
35     3367857
36      666916
37     8444418
38     3909420
39     2297972
40          23
dtype: int64

In [266]:
action

[109038, 666856]

In [270]:
recommend

13    [3400638, 1253714, 3367857, 5100540, 1088584, ...
15    [55109, 129343, 54824, 2297972, 109014, 125734...
Name: impressions, dtype: object

In [283]:
recommend.iloc[0]

[3400638,
 1253714,
 3367857,
 5100540,
 1088584,
 666916,
 54833,
 2922310,
 9711560,
 109038,
 666856,
 10077318,
 1431482,
 129343,
 6339822,
 6806806,
 1041528,
 109013,
 3909420,
 55088,
 3095758,
 109018,
 54885,
 1257342,
 2595006]

In [288]:
pd.Series([item_list[i] for i in a]).loc[pd.Series([item_list[i] for i in a]).isin(recommend.iloc[0])] # 이때 구매한것 #109038

0       109038
2        54885
4      1041528
5       666856
6      6339822
7      9711560
9      1253714
12     3095758
13     2922310
15     1257342
16     1088584
17      109018
18       54833
19     5100540
20    10077318
24      109013
26       55088
27     1431482
28      129343
30     3400638
32     6806806
34     2595006
35     3367857
36      666916
38     3909420
dtype: int64

## 정리

In [345]:
# action : 상호작용
# recommend : 추천목록들
# item_list : 상호작용 + 추천목록들


action = list(map(int,small_train.loc[small_train['action_type'] =='interaction item image'].reference.value_counts(sort=True).index.tolist()))#  # 행위
recommend = small_train.loc[small_train.impressions.notnull()].impressions.apply(lambda x : list(map(int,string_to_array(x))))  # 추천목록

item_list = action  + recommend.iloc[0] + recommend.iloc[1] 
item_list = list(set(item_list))
item_list.sort()   # 정렬 필요!!!

In [348]:
x = df_item.loc[df_item['item_id'].isin(item_list)].set_index('item_id').sort_index().drop(['properties'], axis = 1) # 인덱스 정렬!!!
X = np.nan_to_num(x)
sim_practice = cosine_similarity(X)

In [355]:
np.argsort(-sim_practice)[10]


array([10, 25,  3,  1, 16, 12, 34, 39, 21, 19,  8, 18, 29, 28, 24, 20, 17,
        9,  2, 33, 40,  5,  0, 14,  7, 38,  4, 22, 11,  6, 31, 35, 36, 15,
       27, 30, 13, 37, 32, 26, 23], dtype=int64)

In [360]:
# 상호작용 한 것 [109038, 666856] (앎)
# 구매한 것 [109038, 1257342] (모름)



# 상호작용 젤많이한 109038 을 기준으로 추천하기
item_list.index(109038)  # 10 
sim_idx = list(np.argsort(-sim_practice)[10])  # 109038 과 유사한 순서대로 아이템 인덱스리스트
sim_item_id = [item_list[i] for i in sim_idx]  # 109038 과 유사한 순서대로 아이템 아이디

# 바로 윗줄 일반화
sim_idx = list(np.argsort(-sim_practice)[item_list.index(action[0])])  # 상호작용 제일 많이된 아이템과 유사한 순서대로 아이템 인덱스리스트
sim_item_id = [item_list[i] for i in sim_idx]  #  상호작용 제일 많이된 아이템과 유사한 순서대로 아이템 아이디

print(sim_idx)
print(sim_item_id)
print(len(sim_idx))
print(len(sim_item_id))

# # 첫번째 구매할 때 추천목록
# pd.Series(sim_item_id).loc[pd.Series(sim_item_id).isin(recommend.iloc[0])] #정답 109038
# # 두번째 구매할 때 추천목록
# pd.Series(sim_item_id).loc[pd.Series(sim_item_id).isin(recommend.iloc[1])] #정답 1257342

[10, 25, 3, 1, 16, 12, 34, 39, 21, 19, 8, 18, 29, 28, 24, 20, 17, 9, 2, 33, 40, 5, 0, 14, 7, 38, 4, 22, 11, 6, 31, 35, 36, 15, 27, 30, 13, 37, 32, 26, 23]
[109038, 2237222, 54885, 54824, 1041528, 666856, 6339822, 9711560, 1332971, 1253714, 109014, 1166793, 3095758, 2922310, 2050977, 1257342, 1088584, 109018, 54833, 5100540, 10077318, 55091, 54804, 666936, 109013, 9132132, 55088, 1431482, 129343, 55109, 3400638, 6622154, 6806806, 1031578, 2595006, 3367857, 666916, 8444418, 3909420, 2297972, 1474297]
41
41


In [361]:
# 첫번째 구매할 때 추천목록
pd.Series(sim_item_id).loc[pd.Series(sim_item_id).isin(recommend.iloc[0])] #정답 109038

0       109038
2        54885
4      1041528
5       666856
6      6339822
7      9711560
9      1253714
12     3095758
13     2922310
15     1257342
16     1088584
17      109018
18       54833
19     5100540
20    10077318
24      109013
26       55088
27     1431482
28      129343
30     3400638
32     6806806
34     2595006
35     3367857
36      666916
38     3909420
dtype: int64

In [362]:
# 두번째 구매할 때 추천목록
pd.Series(sim_item_id).loc[pd.Series(sim_item_id).isin(recommend.iloc[1])] #정답 1257342

1      2237222
2        54885
3        54824
8      1332971
10      109014
11     1166793
14     2050977
15     1257342
17      109018
18       54833
20    10077318
21       55091
22       54804
23      666936
25     9132132
26       55088
28      129343
29       55109
31     6622154
33     1031578
36      666916
37     8444418
38     3909420
39     2297972
40     1474297
dtype: int64